In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
import json
import os
import dateparser
import urllib.parse
from openai import OpenAI
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromiumService
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType
from selenium.webdriver.support.ui import WebDriverWait

#from langchain.utilities import GoogleSearchAPIWrapper
with open("./keys.json", 'r') as j:
    keys = json.loads(j.read())
    os.environ["OPENAI_API_KEY"] = keys["OPENAI_API_KEY"]
    os.environ["GOOGLE_API_KEY"] = keys["GOOGLE_API_KEY"]


In [2]:
def scrap_text_bs4(url: str) -> str:
    """
    get all the text of a webpage from the url
    """
    headers = { 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0' }
    html = requests.get(url, headers=headers).text
    soup = BeautifulSoup(html, features="html.parser")
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    return '\n'.join(line for line in lines if line)

def scrap_text_selenium(url: str) -> str:
    driver = webdriver.Chrome(service=ChromiumService(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()))
        # Setup Selenium WebDriver
    # Open the webpage
    driver.get(url)

    # Wait for JavaScript to load (if necessary)
    WebDriverWait(driver, 10).until(lambda d: d.execute_script('return jQuery.active == 0'))

    # Extract all text from the webpage
    text = driver.find_element(By.TAG_NAME, 'body').text
    driver.quit()

    return text


In [3]:
scrap_text_selenium("https://sansrefrain.fr/agenda-des-sorties-albums-et-singles-rap-francais/")

'CRÉER MON COMPTE\nALBUMSARTISTESCOMMUNAUTÉVENTES\nALBUMS À L’AFFICHE\nMEILLEURS ALBUMS\nAGENDA DES SORTIES\nANNIVERSAIRES\nTOUS LES ALBUMS\nSansRefrain Albums Agenda des sorties rap français\nAgenda des sorties rap français\nToutes les sorties rap français de la semaine.\nAbonnez-vous à un artiste pour être notifié de ses sorties albums et singles\nSemaine du 13 novembre 2023\n17\nNOV\nCASINO\nLumjr\n17\nNOV\nKAIJU: MECHA\nDouble Zulu & Pablaw\n17\nNOV\nera 24\nRonisia\n1,5\n17\nNOV\nAmbition\nGuy2Bezbar\n1\n17\nNOV\nRENÉGAT\nMvms\n17\nNOV\nCYNE\nGEAVN\n15\nNOV\nFenêtre ouverte\nPeka59\n15\nNOV\nMaintenant ou jamais\nUrde\n14\nNOV\nCoffret ultime\nLorenzo\n13\nNOV\nSorcellerie\nWarlock\nNotez. Critiquez. Soutenez.\nMeilleurs albums de tous les temps\nMeilleures ventes albums par semaine\nProchaines sorties albums\nAlbums anniversaires de la semaine\nMeilleurs artistes rap français\nArtistes les plus soutenus\nCommunauté\nLe projet SansRefrain\nSoutenir un artiste\nFAQ\nPolitique de co

In [7]:
def google_search_playlists(query, n=3):
    query = urllib.parse.quote(query)
    url = f"https://customsearch.googleapis.com/customsearch/v1?cx={keys['GOOGLE_CSE_ID_SPOTIFY']}&q={query}&key={keys['GOOGLE_API_KEY']}"
    resp = requests.get(url).json()
    print(resp)
    return [s["link"] for s in resp["items"][0:n]]


def google_search_image(query):
    query = urllib.parse.quote(query)
    url = f"https://customsearch.googleapis.com/customsearch/v1?cx={keys['GOOGLE_CSE_ID_IMAGE']}&imgSize=LARGE&imgType=photo&q={query}&searchType=image&key={keys['GOOGLE_API_KEY']}"
    resp = requests.get(url).json()
    return resp["items"][0]["link"]
    

In [9]:
google_search_playlists("Shay, Pourvu qu'il pleuve")

{'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&relatedSite={relatedSite?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': "Google Custom Search - Shay, Pourvu qu'il pleuve", 'searchTerms': "Shay, Pourvu qu'il pleuve", 'count': 10, 'startIndex': 1, 'inputEncoding': 'utf8', 'outputEncoding': 'utf8', 'safe': 'off', 'cx': 

KeyError: 'items'

In [5]:
google_search_image("freeze corleone l'attaque des clone")

'https://i1.sndcdn.com/artworks-GJ73UENoSrz2oSQc-A6ijyA-t500x500.jpg'

In [17]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_album_list_from_text",
            "description": "Get a list of album releases from a text coming from a webpage",
            "parameters": {
                "type": "object",
                "properties": {
                    "albums": {
                        "type": "array",
                        "description": "The name of the released albums",
                        "items": {
                            "type": "object",
                            "properties": {
                                "title": {
                                    "type": "string",
                                    "description": "The title of the album",
                                },
                                "artist": {
                                    "type": "string",
                                    "description": "The artist of the album",
                                },
                                "date": {
                                    "type": "string",
                                    "description": "The release date of the album. The date format should be 'day month year' without a comma, for example, '21 May 2023'. Add the current year if no year is supplied",
                                },
                            }
                        }
                    }
                },
                "required": ["albums"],
            },
        },
    }
]


client = OpenAI()
def get_album_list_from_url(url, model="gpt-3.5-turbo-1106", scraper="bs4"):
    """
    """
    if scraper=="bs4":
        text = scrap_text_bs4(url)
    elif scraper=="selenium":
        text = scrap_text_selenium(url)
    message = f"""
    Can you get a list of all the unreleased album from the following text, for each album give the the title, the artist and the relase date, sort album by descending date. Unreleased means that the release date is after {datetime.datetime.now().year}, please respond in json format. {text}
    """
    messages = [{"role": "user", "content": f"{message}"}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        response_format={ "type": "json_object" },
        tools=TOOLS,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    return json.loads(response.choices[0].message.tool_calls[0].function.arguments)["albums"]

def string_to_timestamp(string, format):
    return datetime.datetime.strptime(string, format)

def create_playlist_title_from_album(album, model="gpt-3.5-turbo-1106"):
    message = f"""creé un titre court pour une playlist spotify qui porte sur l'album : {album["title"]}, et l'artiste {album["artist"]}"""
    messages = [{"role": "user", "content": f"{message}"}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    return response.choices[0].message.content

def create_playlist_description_from_album(album, model="gpt-3.5-turbo-1106"):
    message  = f"""Crée un description courte d'une phrase maximum pour un playlist spotify qui porte sur l'album : {album["title"]}, et l'artiste {album["artist"]}"""
    messages = [{"role": "user", "content": f"{message}"}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    return response.choices[0].message.content

def create_upcoming_release_playlist_candidates_from_url(url, model="gpt-3.5-turbo-1106", scraper="bs4"):
    playlist_candidates= []
    album_list = get_album_list_from_url(url, model=model, scraper=scraper)
    print(album_list)
    unreleased_album_list = parse_album_list(album_list)
    for a in unreleased_album_list:
        title = create_playlist_title_from_album(a)
        description= create_playlist_description_from_album(a)
        print(f"{a['artist']}, {a['title']} playlist".encode('utf-8').decode('unicode_escape'))
        influences = google_search_playlists(f"{a['artist']}, {a['title']} playlist".encode('utf-8').decode('unicode_escape'))
        image = google_search_image(f"{a['artist']}, {a['title']}")
        playlist_candidates.append({"title": title, "description": description, "influences":",".join(influences), "additional infos":f"Playlist created for release {a['title']} by {a['artist']}", "image": image})
    return playlist_candidates
    
    
def parse_album_list(album_list):
    album_list_with_parsed_date = []
    for a in album_list:
        date = dateparser.parse(a["date"])
        a["date"] = date
        # keep only parsable date
        if date:
            album_list_with_parsed_date.append(a)
    unreleased_album_list = [a for a in album_list_with_parsed_date if a["date"]>datetime.datetime.today()]
    return unreleased_album_list


In [18]:
#create_upcoming_release_playlist_candidates_from_url("https://sansrefrain.fr/agenda-des-sorties-albums-et-singles-rap-francais/", model="gpt-4-1106-preview", scraper="selenium")
create_upcoming_release_playlist_candidates_from_url("https://www.trackmusik.fr/media/albums-rap-francais")

[{'title': 'Ma vie', 'artist': "l'Allemand ", 'date': '8 décembre 2023'}, {'title': 'LA ROUTE EST LONGUE', 'artist': 'JuL', 'date': '8 décembre 2023'}, {'title': 'La symphonie des chargeurs, vol. 2', 'artist': 'Fababy', 'date': '24 novembre 2023'}, {'title': 'Le fils de la gardienne', 'artist': 'Rémy', 'date': '24 novembre 2023'}, {'title': 'Bitume', 'artist': 'Rocé', 'date': '17 novembre 2023'}, {'title': 'Banlieusards 2', 'artist': 'Kery James', 'date': '17 novembre 2023'}, {'title': 'Années Sauvages Part. 2', 'artist': 'Georgio', 'date': '10 novembre 2023'}, {'title': 'HLM3', 'artist': 'Mister You', 'date': '10 novembre 2023'}, {'title': 'Eau de source', 'artist': 'Souffrance', 'date': '10 novembre 2023'}, {'title': 'Sale dance - Single', 'artist': 'DTF', 'date': '9 novembre 2023'}, {'title': 'SALE ÉPOQUE - EP', 'artist': 'Lacrim', 'date': '8 novembre 2023'}, {'title': 'PKS 1 (Ça pue sa mère) - Single', 'artist': 'Fresh Lapeufra', 'date': '3 novembre 2023'}, {'title': 'Akatsuki - Si

KeyError: 'items'

In [7]:
create_upcoming_release_playlist_candidates_from_url("https://www.billboard.com/lists/new-albums-2023-calendar-new-music-releases-this-year/june-3/")
https://sansrefrain.fr/agenda-des-sorties-albums-et-singles-rap-francais/

1978
2299
[{'title': 'Various artists - Yellowjackets season two soundtrack', 'artist': 'Various artists', 'date': datetime.datetime(2023, 9, 1, 0, 0)}, {'title': 'Olivia Rodrigo - GUTS', 'artist': 'Olivia Rodrigo', 'date': datetime.datetime(2023, 9, 8, 0, 0)}, {'title': 'V - Layover', 'artist': 'V', 'date': datetime.datetime(2023, 9, 8, 0, 0)}, {'title': 'Demi Lovato - Revamped', 'artist': 'Demi Lovato', 'date': datetime.datetime(2023, 9, 15, 0, 0)}, {'title': 'Diddy - The Love Album: Off the Grid', 'artist': 'Diddy', 'date': datetime.datetime(2023, 9, 15, 0, 0)}, {'title': 'Kelly Clarkson - Chemistry (Deluxe)', 'artist': 'Kelly Clarkson', 'date': datetime.datetime(2023, 9, 22, 0, 0)}, {'title': 'Kylie Minogue - Tension', 'artist': 'Kylie Minogue', 'date': datetime.datetime(2023, 9, 22, 0, 0)}, {'title': "Taylor Swift - 1989 (Taylor's Version)", 'artist': 'Taylor Swift', 'date': datetime.datetime(2023, 10, 27, 0, 0)}, {'title': 'Various artists - A Tribute to The Judds', 'artist': 'Va

[{'title': '"Yellowjackets S2 OST: Eclectic Vibes"',
  'description': 'La playlist ultime pour les fans de la série Yellowjackets saison deux, avec la bande originale captivante interprétée par divers artistes.',
  'influences': 'https://open.spotify.com/track/3L2LHQIzwLshvWKz07NZv5,https://open.spotify.com/album/1khiHkdRH0RX3dOf2ZTKXY,https://open.spotify.com/album/44MoiIXWGc5WhwB7AJwnCB',
  'additional infos': 'Playlist created for release Various artists - Yellowjackets season two soundtrack by Various artists',
  'image': 'https://img.broadtime.com/418467253006:500.webp'},
 {'title': '"GUTS: Olivia Rodrigo\'s Musical Journey"',
  'description': '"Plongez dans l\'univers émotionnellement puissant d\'Olivia Rodrigo avec son album GUTS, où sa voix captivante et ses paroles sincères vous transportent dans un voyage introspectif."',
  'influences': 'https://open.spotify.com/playlist/4aurFBWbfxIOae2vkkdAP9,https://open.spotify.com/artist/1McMsnEElThX1knmY4oliG,https://open.spotify.com/pl